In [1]:
import paramiko
import json
import configparser

In [2]:
multi_line_mongo_servers = """
#activities cluster
192.168.38.238
192.168.38.236
192.168.42.205
192.168.38.237
172.24.77.34
172.24.77.35

#coupan cluster
192.168.38.233
192.168.38.234
192.168.38.235

#users cluster
192.168.38.230
192.168.38.231
192.168.38.232
172.24.81.74
172.24.81.75
172.24.80.27
"""

In [13]:
ssh_config_file_loc = '/Users/yash.dalmia/spam_filter/jupyter_notebooks/config/config.ini'

config_ssh = configparser.ConfigParser()
config_ssh.read(ssh_config_file_loc)

username=config_ssh['LOGIN']['username']
password=config_ssh['LOGIN']['password']

In [5]:
def check_jsso_whitelisting(ssh_client):
    """
    checks if the server is jsso whitelisted or not
    param : 
    ssh_client - paramiko.SSHClient() type
    """
    try:
        cmd = "curl 'https://jsso.indiatimes.com/sso/services/ip/get'"
        
        stdin, stdout, stderr = ssh_client.exec_command(cmd)
        
        out = ''.join(stdout.readlines())
#         print(out)
        
        a = json.loads(out)
        if (a['status']=='SUCCESS'):
            print('server is jsso whitelisted')
        else:
            print('server is NOT jsso whitelisted')
    except Exception as e:
        print(e)

In [6]:
def check_mongo_whitelisting(ssh_client, mongo_server):
    """
    checks if the mongo server is whitelisted for the server or not
    param : 
    ssh_client - paramiko.SSHClient() type
    """
    try:
        port = 27017
        cmd = "telnet {} {}".format(mongo_server, port)
        stdin, stdout, stderr = ssh_client.exec_command(cmd)
        stdin.write('''
                quit
                ''')
        out_resp = stdout.readlines()
#         print(out_resp)
        if(len(out_resp)>1 and out_resp[1].startswith('Connected to')):
            """
            standard response when whitelisted - ['Trying 192.168.34.158...\r\n', 'Connected to 192.168.34.158.\r\n', 
            "Escape character is '^]'.\r\n"]
            """
            print('mongo server {:15} is whitelisted for the given server'.format(mongo_server))
        else:
            """
            standard response when not whitelisted - ['Trying 192.168.34.159...\r\n']
            """
            print('mongo server {:15} is NOT whitelisted for the given server'.format(mongo_server))
    except Exception as e:
        print(e)

In [7]:
def get_list_from_multi_line_string(multi_line_string):
    """
    returns a list of string from a multi_line_comment excluding comments (those starting with # and blank lines)
    """
    list_all = list()
    for server in multi_line_string.split("\n"):
        if(server and not server.startswith('#')):
            list_all.append(server)
    return list_all

In [8]:
list_mongo_servers = get_list_from_multi_line_string(multi_line_mongo_servers)

In [12]:
server_ip = "192.168.36.98"

print('server ip : {}\n'.format(server_ip))

with paramiko.SSHClient() as ssh_client:
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(hostname=server_ip,username=username ,password=password)
    check_jsso_whitelisting(ssh_client)
    print()
    for mongo_server in list_mongo_servers:
        check_mongo_whitelisting(ssh_client, mongo_server)

server ip : 192.168.36.98

server is jsso whitelisted

mongo server 192.168.38.238  is whitelisted for the given server
mongo server 192.168.38.236  is whitelisted for the given server
mongo server 192.168.42.205  is whitelisted for the given server
mongo server 192.168.38.237  is whitelisted for the given server
mongo server 172.24.77.34    is whitelisted for the given server
mongo server 172.24.77.35    is whitelisted for the given server
mongo server 192.168.38.233  is whitelisted for the given server
mongo server 192.168.38.234  is whitelisted for the given server
mongo server 192.168.38.235  is whitelisted for the given server
mongo server 192.168.38.230  is whitelisted for the given server
mongo server 192.168.38.231  is whitelisted for the given server
mongo server 192.168.38.232  is whitelisted for the given server
mongo server 172.24.81.74    is whitelisted for the given server
mongo server 172.24.81.75    is whitelisted for the given server
mongo server 172.24.80.27    is whi

In [10]:
# err = ''.join(stderr.readlines())
# out = ''.join(stdout.readlines())
# final_output = str(out)+str(err)
# print(final_output)